# Scoring Structured Output Responses with TLM

This tutorial demonstrates how to score the trustworthiness of structred outputs (JSON etc) using TLM.


## Setup

This tutorial requires an API key for an LLM provider. Some possibilities include: `OPENAI_API_KEY`, GEMINI_API_KEY, DEEPSEEK_API_KEY, AWS_ACCESS_KEY_ID/AWS_SECRET_ACCESS_KEY, etc.

The TLM Python client can be installed using pip:

In [ ]:
%pip install --upgrade trustworthy-llm

In [ ]:
# Set your API key
import os

os.environ["OPENAI_API_KEY"] = "<INSERT_API_KEY>"  # or other LLM provider API key

In [3]:
from pydantic import create_model
from typing import Optional

from tlm import TLM

## PII Extraction Use Case

This tutorial showcases a PII (Personally Identifiable Information) extraction example.

Each text sample contains various types of personal information embedded within natural language text. The task is to extract different categories of PII from the text. Each example contains multiple types of PII that need to be identified and classified into specific categories including names (FIRSTNAME, LASTNAME), dates (DATE), and account numbers (ACCOUNTNUMBER).

Let’s take a look at a few samples below:

In [4]:
input_texts = [
    "Melvin, the password of your study support account has been changed to Xnjv7nCydECf for security purposes. Please update it promptly.",
    "In relation to the filed litigation, we hereby request full disclosure of all data logs associated with 242.218.157.166 and cd9f:d9e5:1ceb:fd39:b2d7:f3fd:c9cd:c27b tied to the account of Fleta London Emard. This involves her employment account Investment Account with Gutkowski Inc.",
    "We would like to do a follow-up meeting with Sierra Green regarding her recent surgery. The proposed date is August 13, 2013 at our clinic in West Nash.", 
    "Melvin, the password of your study support account has been changed to Xnjv7nCydECf for security purposes. Please update it promptly.", 
    "Is your business tax-ready? Our team in Novato is here to help you navigate through Martinique's complex tax rules. Contact us at 56544500.", 
    "To: Maximillian Noah Moore, we forgot to update your record with phone IMEI: 30-265288-033265-8. Could you please provide it in your earliest convenience to keep your records updated.", 
]

print(input_texts[0])

Melvin, the password of your study support account has been changed to Xnjv7nCydECf for security purposes. Please update it promptly.


## Obtain LLM Predictions

### Define Structured Output Schema

We know that the 4 PII fields that we want to extract are: `['FIRSTNAME', 'LASTNAME', 'DATE', 'ACCOUNTNUMBER']`

Using that, we can create a Pydantic model to represent our PII extraction schema. Each field is optional and can be None if that entity type is not found in the text:



In [5]:
pii_entities = ['FIRSTNAME', 'LASTNAME', 'DATE', 'ACCOUNTNUMBER']
fields = {name: (Optional[str], None) for name in pii_entities}

PII = create_model("PII", **fields)

### Prompt TLM for responses and trust scores

In [ ]:
tlm = TLM()  

sample_text = input_texts[0]
openai_kwargs = {
    "model": "gpt-4.1-mini", 
    "messages": [{"role": "user", "content": f"Extract PII information from the following text, return null if the entity is not found: {sample_text}"}],
    "response_format": PII,
}
tlm_result = tlm.create(**openai_kwargs)

The returned object matches what any LLM would ordinarily return, except it has an additional `trustworthiness_score` field from TLM with extra information like the trustworthiness score. 

In [ ]:
print("LLM response: ", tlm_result["response"].choices[0].message.parsed)
print("Trustworthiness score: ", tlm_result["trustworthiness_score"])

LLM response: FIRSTNAME='Melvin' LASTNAME=None DATE=None ACCOUNTNUMBER=None
Trustworthiness score: 0.98902
